# Neural Network Testing with less noise Data

### Get Libraries and Functions

In [1]:
import sys 
sys.path.append("../")

import torch 
import torch.nn

from Conv1d_NN import Conv1d_NN

from models import * 
from train import * 
from dataset import * 



The Arguments for Data are: 
num_samples: 5000 
 train_split: 0.8 
 template_len: 12 
 padding: [36,60] 
 scale_coeff: .4 
 max_translation: 48 
 corr_noise_scale: 0.25 
 iid_noise_scale: 2e-2 
 shear_scale: 0.75 
 shuffle_seq: False 
 final_seq_length: 40 
 seed: 42


The Arguments for Model are: 
input_size: 40 
 output_size: 10 
 hidden_size: 256 
 learning_rate: 1e-2 
 weight_decay: 0 
 batch_size: 100 
 total_steps: 6000 
 print_every: 1000 
 eval_every: 250 
 checkpoint_every: 1000 
 device: mps 
 seed: 42


### Less Noise Data

In [2]:
less_noise_dataset = MNIST1D_Dataset()
less_noise_dataset.data_args.corr_noise_scale = 0.1
less_noise_dataset.data_args.iid_noise_scale = 0.01

less_noise_data = less_noise_dataset.make_dataset()


The Arguments for Data are: 
num_samples: 5000 
 train_split: 0.8 
 template_len: 12 
 padding: [36,60] 
 scale_coeff: .4 
 max_translation: 48 
 corr_noise_scale: 0.25 
 iid_noise_scale: 2e-2 
 shear_scale: 0.75 
 shuffle_seq: False 
 final_seq_length: 40 
 seed: 42


The Arguments for Model are: 
input_size: 40 
 output_size: 10 
 hidden_size: 256 
 learning_rate: 1e-2 
 weight_decay: 0 
 batch_size: 100 
 total_steps: 6000 
 print_every: 1000 
 eval_every: 250 
 checkpoint_every: 1000 
 device: mps 
 seed: 42


### Regular Conv1d_NN

In [8]:
# Regular Conv1d_NN
model1 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =32, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1280, 10)
)

# Get summary 
from torchsummary import summary
summary(model1, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 40]           3,104
         Conv1d_NN-5               [-1, 32, 40]               0
              ReLU-6               [-1, 32, 40]               0
            Conv1d-7               [-1, 32, 40]           3,104
         Conv1d_NN-8               [-1, 32, 40]               0
              ReLU-9               [-1, 32, 40]               0
          Flatten-10                 [-1, 1280]               0
           Linear-11                   [-1, 10]          12,810
Total params: 19,146
Trainable params: 19,146
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [10]:
# Regular Conv1d_NN Results 
print("Regular Conv1d_NN Results")
results1 = train_model(less_noise_data, model1, less_noise_dataset.model_args)

Regular Conv1d_NN Results
step 1000, dt 18.78s, train_loss 1.775e-01, test_loss 1.864e+00, train_acc 87.0, test_acc 58.6
step 2000, dt 17.75s, train_loss 1.711e-01, test_loss 1.909e+00, train_acc 86.5, test_acc 57.8
step 3000, dt 16.69s, train_loss 2.677e-01, test_loss 1.961e+00, train_acc 85.6, test_acc 57.9
step 4000, dt 15.23s, train_loss 2.795e-01, test_loss 1.942e+00, train_acc 90.3, test_acc 59.7
step 5000, dt 15.12s, train_loss 2.220e-01, test_loss 2.083e+00, train_acc 89.2, test_acc 58.6
step 6000, dt 18.95s, train_loss 1.782e-01, test_loss 2.268e+00, train_acc 87.0, test_acc 56.9


### Local -> Global Neural Network

In [11]:
# Local -> Global Conv1d_NN
model2 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model2, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 38]             128
              ReLU-2               [-1, 32, 38]               0
            Conv1d-3               [-1, 32, 38]           3,104
         Conv1d_NN-4               [-1, 32, 38]               0
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 36]           3,104
              ReLU-7               [-1, 32, 36]               0
            Conv1d-8               [-1, 32, 36]           3,104
         Conv1d_NN-9               [-1, 32, 36]               0
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 34]           3,104
             ReLU-12               [-1, 32, 34]               0
           Conv1d-13               [-1, 32, 34]           3,104
        Conv1d_NN-14               [-1,

In [12]:
# Local -> Global Conv1d_NN Results 
print("Local -> Global Conv1d_NN Results")
results2 = train_model(less_noise_data, model2, less_noise_dataset.model_args)

Local -> Global Conv1d_NN Results
step 1000, dt 19.05s, train_loss 2.282e-01, test_loss 1.826e-01, train_acc 93.5, test_acc 93.7
step 2000, dt 16.71s, train_loss 5.495e-02, test_loss 1.525e-01, train_acc 96.2, test_acc 95.0
step 3000, dt 17.16s, train_loss 5.983e-02, test_loss 1.198e-01, train_acc 96.8, test_acc 95.9
step 4000, dt 17.11s, train_loss 1.013e-01, test_loss 2.066e-01, train_acc 95.5, test_acc 94.7
step 5000, dt 17.63s, train_loss 8.331e-02, test_loss 1.128e-01, train_acc 97.1, test_acc 96.7
step 6000, dt 19.91s, train_loss 3.317e-02, test_loss 1.029e-01, train_acc 97.7, test_acc 96.3


### Global -> Local Neural Network

In [13]:
# Global -> Local Conv1d_NN
model3 = nn.Sequential(
   Conv1d_NN(in_channels=1, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model3, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 38]           3,104
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 38]           3,104
         Conv1d_NN-7               [-1, 32, 38]               0
              ReLU-8               [-1, 32, 38]               0
            Conv1d-9               [-1, 32, 36]           3,104
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 36]           3,104
        Conv1d_NN-12               [-1, 32, 36]               0
             ReLU-13               [-1, 32, 36]               0
           Conv1d-14               [-1,

In [14]:
# Global -> Local Conv1d_NN Results 
print("Global -> Local Conv1d_NN Results")
results3 = train_model(less_noise_data, model3, less_noise_dataset.model_args)

Global -> Local Conv1d_NN Results
step 1000, dt 19.20s, train_loss 3.021e-01, test_loss 3.101e-01, train_acc 90.5, test_acc 88.4
step 2000, dt 20.50s, train_loss 1.497e-01, test_loss 2.262e-01, train_acc 94.6, test_acc 93.3
step 3000, dt 20.32s, train_loss 1.508e-01, test_loss 1.850e-01, train_acc 96.7, test_acc 95.0
step 4000, dt 18.34s, train_loss 5.525e-02, test_loss 1.725e-01, train_acc 97.1, test_acc 94.3
step 5000, dt 20.50s, train_loss 2.664e-02, test_loss 1.399e-01, train_acc 97.7, test_acc 95.9
step 6000, dt 20.56s, train_loss 4.173e-02, test_loss 1.984e-01, train_acc 97.6, test_acc 93.6


### Branching Neural Network

In [3]:
model4 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 3), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 3),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 3),
   nn.Flatten(), 
   nn.Linear(160, 10)
)


# Get summary
from torchsummary import summary 
summary(model4, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 38]              64
              ReLU-2               [-1, 16, 38]               0
            Conv1d-3               [-1, 16, 40]              64
         Conv1d_NN-4               [-1, 16, 40]               0
              ReLU-5               [-1, 16, 40]               0
            Conv1d-6               [-1, 16, 40]             528
  BranchingNetwork-7               [-1, 16, 40]               0
            Conv1d-8                [-1, 8, 38]             392
              ReLU-9                [-1, 8, 38]               0
           Conv1d-10                [-1, 8, 40]             392
        Conv1d_NN-11                [-1, 8, 40]               0
             ReLU-12                [-1, 8, 40]               0
           Conv1d-13                [-1, 8, 40]             136
 BranchingNetwork-14                [-1

In [4]:
# Branching Network Results 
print("Branching Conv1d_NN Conv1d Results")
results4 = train_model(less_noise_data, model4, less_noise_dataset.model_args)

Branching Conv1d_NN Conv1d Results
step 1000, dt 26.52s, train_loss 9.713e-02, test_loss 1.964e-01, train_acc 96.8, test_acc 94.5
step 2000, dt 23.45s, train_loss 1.110e-03, test_loss 4.911e-02, train_acc 99.7, test_acc 98.6
step 3000, dt 23.29s, train_loss 3.036e-02, test_loss 4.941e-02, train_acc 99.2, test_acc 98.7
step 4000, dt 23.32s, train_loss 1.128e-01, test_loss 7.441e-02, train_acc 99.0, test_acc 97.8
step 5000, dt 21.88s, train_loss 2.627e-05, test_loss 2.548e-02, train_acc 100.0, test_acc 99.3
step 6000, dt 23.11s, train_loss 1.709e-05, test_loss 1.119e-02, train_acc 100.0, test_acc 99.6
